In [16]:
import torch
import torch.nn as nn
import numpy as np

mask_rate = 0.115
patch_len = 4
stride = 4
b=1
s = 96
c=7
x = torch.rand(b,s,c)

def generate_binomial_mask(B, T, C=None, p=0.5):
    if C:
        return torch.from_numpy(np.random.binomial(1, 1 - p, size=(B, T, C))).to(torch.bool)
    else:
        return torch.from_numpy(np.random.binomial(1, 1 - p, size=(B, T))).to(torch.bool)
def expand_tensor(input_tensor, third_dim_size):
    # 将输入张量转换为三维张量
    expanded_tensor = input_tensor.unsqueeze(2).expand(-1, -1, third_dim_size)

    return expanded_tensor.bool()

def patch_mask(x, mask_ratio, patch_len=12, stride=12):
    px = x.clone().permute(0, 2, 1)

    padding_patch_layer = nn.ReplicationPad1d((0, stride))
    px = padding_patch_layer(px)
    px = px.unfold(dimension=-1, size=patch_len, step=stride)
    px = torch.reshape(px, (px.shape[0] * px.shape[1], px.shape[2], px.shape[3]))

    mask = generate_binomial_mask(px.size(0), px.size(1), p=mask_ratio).to(x.device)

    return mask



def patch_mask(x, mask_ratio, patch_len=12, stride=12):
    px = x.clone().permute(0, 2, 1)

    padding_patch_layer = nn.ReplicationPad1d((0, stride))
    px = padding_patch_layer(px)
    px = px.unfold(dimension=-1, size=patch_len, step=stride)
    px = torch.reshape(px, (px.shape[0] * px.shape[1], px.shape[2], px.shape[3]))

    mask = generate_binomial_mask(px.size(0), px.size(1), p=mask_ratio).to(x.device)

    return mask

mask = patch_mask(x, mask_rate, patch_len, stride)
mask = expand_tensor(mask, patch_len)
mask = mask.reshape(b, c, -1)[:, :, :s].permute(0, 2, 1)

In [17]:
mask.int()

tensor([[[1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 0, 1, 1, 1, 1, 1],
         [1, 0, 1, 1, 1, 1, 1],
         [1, 0, 1, 1, 1, 1, 1],
         [1, 0, 1, 1, 1, 1, 1],
         [1, 1, 0, 1, 1, 1, 1],
         [1, 1, 0, 1, 1, 1, 1],
         [1, 1, 0, 1, 1, 1, 1],
         [1, 1, 0, 1, 1, 1, 1],
         [0, 1, 1, 1, 1, 1, 1],
         [0, 1, 1, 1, 1, 1, 1],
         [0, 1, 1, 1, 1, 1, 1],
         [0, 1, 1, 1, 1, 1, 1],
         [1, 1, 0, 1, 1, 1, 0],
         [1, 1, 0, 1, 1, 1, 0],
         [1, 1, 0, 1, 1, 1, 0],
         [1, 1, 0, 1, 1, 1, 0],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1],
        

In [6]:
x*mask

tensor([[[3.6419e-01, 4.3512e-01, 1.1558e-01, 1.5930e-01, 1.1113e-01,
          7.0491e-01, 9.9028e-01],
         [3.8061e-01, 4.8991e-01, 9.6318e-01, 7.0700e-01, 2.6783e-01,
          5.5377e-01, 2.5595e-01],
         [8.5292e-01, 8.7662e-01, 4.6254e-01, 2.1769e-01, 3.4266e-01,
          6.9115e-01, 1.4197e-01],
         [8.1255e-01, 8.2015e-02, 5.2476e-01, 2.7333e-01, 4.2011e-01,
          5.0631e-01, 3.7496e-01],
         [2.5684e-01, 4.6117e-01, 4.4928e-02, 7.0809e-01, 4.7318e-01,
          1.9954e-02, 9.3633e-01],
         [6.6580e-01, 1.6090e-01, 4.8530e-01, 3.9319e-01, 4.8001e-01,
          9.0201e-01, 5.1212e-01],
         [7.5045e-02, 6.3382e-01, 7.8909e-02, 4.0910e-01, 9.8649e-01,
          1.5817e-01, 9.5040e-04],
         [9.4331e-01, 1.1100e-02, 7.1027e-01, 3.9107e-01, 5.1099e-01,
          6.9964e-01, 8.3117e-01],
         [5.7103e-01, 2.7946e-01, 2.6341e-01, 6.5105e-01, 5.6960e-01,
          4.8794e-01, 4.9425e-01],
         [8.0595e-01, 9.4073e-01, 6.5680e-01, 8.6779e-0